In [1]:
%matplotlib inline

import numpy as np
from numpy import array, arange, cos, exp, pi, zeros, column_stack, ones, newaxis, log, dot, append, zeros_like
from numpy.random import permutation, shuffle, random, randint, rand
from scipy.io import loadmat
from scipy.optimize import minimize, fmin_bfgs

from matplotlib import pyplot as plt
from matplotlib.figure import Figure

from IPython.display import Latex

In [2]:
# Setup the parameters you will use for this exercise
input_layer_size  = 400;  # 20x20 Input Images of Digits
hidden_layer_size = 25;   # 25 hidden units
num_labels = 10;          # 10 labels, from 1 to 10

 ### =========== Part 1: Loading and Visualizing Data =============

In [3]:
handwritten_digits = loadmat('ex4data1.mat')
handwritten_digits.keys()

features = handwritten_digits['X']
m, n = features.shape

org_y = handwritten_digits['y']
y = org_y.copy()
y[y==10] = 0
features.shape, y.shape

((5000, 400), (5000, 1))

### ================ Part 2: Loading Parameters ================

In [6]:
# Loading Saved Neural Network Parameters ...
weight = loadmat('ex3weights.mat')
print(weight.keys())

# Unroll parameters 
t1 = weight['Theta1'].ravel(order='F')
t2 = weight['Theta2'].ravel(order='F')
# nn_params = [Theta1(:) ; Theta2(:)];
print(weight['Theta1'].shape, weight['Theta2'].shape)
nn_params = np.r_[t1, t2]
nn_params.shape

dict_keys(['__header__', '__version__', 'Theta1', '__globals__', 'Theta2'])
(25, 401) (10, 26)


(10285,)

### ================ Part 3: Compute Cost (Feedforward) ================

In [11]:
_lambda = 0
J = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, features, y, _lambda)
print ('Cost at parameters (loaded from ex4weights): ', J,'  \n(this value should be about 0.287629)\n')

Cost at parameters (loaded from ex4weights):  0.287629165161   
(this value should be about 0.287629)



###  ================== Part 4: Implement Regularization ===============

In [12]:
_lambda = 1
J = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, features, y, _lambda)
print('Cost at parameters (loaded from ex4weights): ', J,'  \n(this value should be about 0.383770)\n')

Cost at parameters (loaded from ex4weights):  0.383769859091   
(this value should be about 0.383770)



###  ================ Part 5: Sigmoid Gradient  ================

In [13]:
def sigmoidGradient(z):
    return sigmoid(z) * (1-sigmoid(z))

print('Sigmoid gradient evaluated at [1 -0.5 0 0.5 1]:\n  ')
print(sigmoidGradient(array([1, -0.5, 0, 0.5, 1])))

Sigmoid gradient evaluated at [1 -0.5 0 0.5 1]:
  
[ 0.19661193  0.23500371  0.25        0.23500371  0.19661193]


### ================ Part 6: Initializing Pameters ================

### =============== Part 7: Implement Backpropagation ===============

# Steps for training a Neural Network:

## 1- Randomly initialize weights

We usually initialize the weights to small values close to zero

In [15]:
def randInitializeWeights(L_in_size, L_out_size):
    epsilon_init = np.sqrt(6)/np.sqrt(L_in_size+L_out_size)
    epsilon_init = 0.12
    return  rand(L_out_size, L_in_size+1) * 2*epsilon_init - epsilon_init


initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size);
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels);

print(initial_Theta1.shape, initial_Theta2.shape)

(25, 401) (10, 26)


## 2- Implement forward propagation to get $h_\Theta(x^{(i)})$ for any $x^{(i)}$

In [16]:
def feed_forward(x, Theta1, Theta2):
    z2 = Theta1.dot(x[:,newaxis])
    a2 = np.r_[[[1]], sigmoid(z2)]
    
    z3 = Theta2.dot(a2)
    return sigmoid(z3).ravel()

## 3- Implement code to compute cost function $J(\Theta)$

In [17]:
def sigmoid(z): 
    return 1/(1+exp(-z))

def sigmoidGradient(z):
    return sigmoid(z) * (1-sigmoid(z))

def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, features, y, reg_parameter):
    if nn_params.ndim != 1:
        return
    theta1_size = (input_layer_size+1) * hidden_layer_size
    Theta1 = nn_params[:theta1_size].reshape((hidden_layer_size,input_layer_size+1), order='F') # (25, 401)
    Theta2 = nn_params[theta1_size:].reshape((num_labels, hidden_layer_size+1), order='F') # (10, 26)
    
    m, _ = features.shape
    a_1 = np.c_[ones((m)), features]
    
    z_2 = Theta1.dot(a_1.T) # (25, 401) * (401, 5000)
    a_tmp = sigmoid(z_2)    # (25, 5000)
    
    a_2 = np.vstack((ones((m)), a_tmp))
    z_3 = Theta2.dot(a_2)
    a_3 = sigmoid(z_3)
    
    #ex_sum = 0
    #for i in arange(m):
    #    yVec = zeros((num_labels,1))
    #    yVec[y[i]] = 1
    #    yVec = yVec.ravel()
    #    yVec = np.roll(yVec, -1)
    #    ex_sum = ex_sum+ np.sum(-yVec*np.log(a_3[:,i]) - (1-yVec)*np.log(1 - a_3[:,i]))
    #else:
    #    print ex_sum/m
    
    incidence_y = zeros((y.size, num_labels))
    y_1 = y.ravel()
    
    incidence_y[arange(m), y_1] = 1  # (5000, 10)
    incidence_y = np.roll(incidence_y, -1, axis=1)
    
    reg_term = _lambda *(np.sum(Theta1[:,1:]**2) + np.sum(Theta2[:,1:]**2))/(2*m)
    
    return np.sum(-incidence_y*np.log(a_3.T) - (1-incidence_y)*np.log(1 - a_3.T))/m +reg_term

In [18]:
y.shape

(5000, 1)

## 4- Implement backprop to compute partial derivatives $\frac{\partial}{\partial \Theta_{jk}^{(l)}} J(\Theta)$

In [19]:
def nn_gradient(nn_params, input_layer_size, hidden_layer_size, num_labels,features, y, _lambda):
    m = y.size
    X = np.c_[ones((m)), features]
    
    
    
    if nn_params.ndim != 1:
        return
    theta1_size = (input_layer_size+1) * hidden_layer_size
    Theta1 = nn_params[:theta1_size].reshape((hidden_layer_size,input_layer_size+1), order='F') # (25, 401)
    Theta2 = nn_params[theta1_size:].reshape((num_labels, hidden_layer_size+1), order='F') # (10, 26)

    Delta2 = zeros_like(Theta2)
    Delta1 = zeros_like(Theta1)

    for i in arange(m):
        
        # forward pass
        x = X[i,:]
    
        z2 = Theta1.dot(x[:,newaxis])
        a2 = np.r_[[[1]], sigmoid(z2)]
    
        z3 = Theta2.dot(a2)
        hx = sigmoid(z3).ravel()
    
        # computing the "error terms" that measure how much the nodes were responsible for any errors 
        # in our output
        delta3 = hx - incidence_y[i,:]
        delta2 = Theta2.T.dot(delta3)[1:] * sigmoidGradient(z2).ravel()
    
        Delta2 = Delta2 + delta3[:,newaxis].dot(a2.T)
        Delta1 = Delta1 + delta2[:,newaxis].dot(x[:,newaxis].T)
    
    else:
        D2 = Delta2/m + _lambda/m * np.c_[zeros((Theta2.shape[0])), Theta2[:,1:]]
        D1 = Delta1/m + _lambda/m * np.c_[zeros((Theta1.shape[0])), Theta1[:,1:]]
        return np.r_[D1.ravel(order='F'), D2.ravel(order='F')]

initial_weights = np.r_[initial_Theta1.ravel(order='F'), initial_Theta2.ravel(order='F')]
D = nn_gradient(initial_weights, input_layer_size, hidden_layer_size, num_labels,features, y, _lambda)

# $ \Delta^{(l)} := \Delta^{(l)} + \delta^{(l+1)} (a^{(l)})^T $

## 5- Use gradient checking to compare $\frac{\partial}{\partial \Theta_{jk}^{(l)}} J(\Theta)$ computed using packpropagation vs. using numerical estimate of gradient of $J(\Theta)$ .
Then disable gradient checking code

In [20]:
def gradient_checking(nn_params):
    epsilon = 1e-4
    grad_vect = zeros_like(nn_params)
    for i in arange(nn_params.size):
        e_vector = zeros_like(nn_params)
        
        e_vector[i] = epsilon
        
        plus = nnCostFunction(nn_params+e_vector,input_layer_size, hidden_layer_size, num_labels, features, y, 0)
        minus = nnCostFunction(nn_params-e_vector,input_layer_size, hidden_layer_size, num_labels, features, y, 0)
        grad_estimation = (plus - minus)/(2*epsilon)
        grad_vect[i] = grad_estimation
        
        if i%1000 == 0:
            print(i)
    else:
        return grad_vect
#         print grad_estimation
        
#         if i > 100:
#             print grad_vect
#             break
    
    
G = gradient_checking(initial_weights)



0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [21]:
print(np.c_[D,G][10:])

for diff_value in [1e-10,1e-9,1e-8,1e-7,1e-6,1e-5,1e-4,1e-3,1e-2,1e-1,1]:
    if np.all(np.abs(D-G)<diff_value):
        print ('Andrew Ng says that you should see a relative difference that is less than 1e-9, but you got: ',) 
        print (diff_value)
        break
    else:
        "something wrong"

[[ 0.01089709  0.01089709]
 [-0.00978536 -0.00978536]
 [ 0.016502    0.016502  ]
 ..., 
 [ 0.22879093  0.22879093]
 [ 0.25669864  0.25669864]
 [ 0.25704499  0.25704499]]
Andrew Ng says that you should see a relative difference that is less than 1e-9, but you got: 
1e-10


## 6- Use gradient descent or advanced optimization menthod with backpropagation to try to minimize $J(\Theta)$ as a function of parameters $ \Theta $ 

In [22]:
'''
method : str or callable, optional
Type of solver. Should be one of
    ‘Nelder-Mead’ (see here)
    ‘Powell’ (see here)
    ‘CG’ (see here)
    ‘BFGS’ (see here)
    ‘Newton-CG’ (see here)
    ‘L-BFGS-B’ (see here)
    ‘TNC’ (see here)
    ‘COBYLA’ (see here)
    ‘SLSQP’ (see here)
    ‘dogleg’ (see here)
    ‘trust-ncg’ (see here)
    custom - a callable object (added in version 0.14.0), see below for description.
'''
_lambda = 10
# initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size);
# initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels);


initial_weights = np.r_[initial_Theta1.ravel(order='F'), initial_Theta2.ravel(order='F')]
res = minimize(fun=nnCostFunction, x0 =initial_weights, 
               args=(input_layer_size, hidden_layer_size, num_labels,features, y, _lambda), method='CG', 
               jac=nn_gradient, options={'maxiter':30})

In [23]:
theta1_size = (input_layer_size+1) * hidden_layer_size
opt_Theta1 = res.x[:theta1_size].reshape((hidden_layer_size,input_layer_size+1), order='F') # (25, 401)
opt_Theta2 = res.x[theta1_size:].reshape((num_labels, hidden_layer_size+1), order='F') # (10, 26)

def predict_from_three_layer_NN(Theta1, Theta2, X):
    m, _ = X.shape
    A_1 = np.c_[ones((m)), X] # (5000, 400)
    
    Z_2 = Theta1.dot(A_1.T) # (25, 401) * (401, 5000)
    A_tmp = sigmoid(Z_2).T # (5000, 25)    
    A_2 = np.c_[(ones((m)), A_tmp)] # (5000, 26) 
    
    Z_3 = Theta2.dot(A_2.T) # (10, 26) * (26, 5000) 
    A_3 = sigmoid(Z_3).T # (5000, 10)
    
    return A_3

pred = predict_from_three_layer_NN(opt_Theta1, opt_Theta2, features)
np.mean(pred.argmax(axis=1)+1 == org_y.ravel())*100

92.579999999999998

In [24]:
_lambda

10

# Neural Networks via Keras

In [25]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


In [28]:
print(features.shape)
y

(5000, 400)


array([[0],
       [0],
       [0],
       ..., 
       [9],
       [9],
       [9]], dtype=uint8)

In [99]:
model = Sequential()
# model.add(Dense(4, input_shape=(2,), activation='tanh'))
model.add(Dense(25, input_shape=(400,), activation='sigmoid'))
# model.add(Dense(2, activation='tanh'))
model.add(Dense(10, activation='softmax'))
model.compile(Adam(lr=0.05), 'binary_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_131 (Dense)            (None, 25)                10025     
_________________________________________________________________
dense_132 (Dense)            (None, 10)                260       
Total params: 10,285
Trainable params: 10,285
Non-trainable params: 0
_________________________________________________________________


### The following few lines convert the target vector into an incidence matrix (IN TWO WAYS)

In [69]:
incidence_y = zeros((y.size, num_labels))
y_1 = y.ravel()
   
incidence_y[arange(m), y_1] = 1  # (5000, 10)
# incidence_y = np.roll(incidence_y, -1, axis=1)

## The following method use Keras to do the same job
from keras.utils.np_utils import to_categorical
incidence_y = to_categorical(y)

In [106]:
model.fit(features, incidence_y, epochs=30, verbose=0) # also you could use validation_split=0.1 for development
print(model.evaluate(features,incidence_y))
y_pred = model.predict(features)
np.argmax(y_pred, axis=1)

4096/5000 [=======================>......] - ETA: 0s[0.0037235659333663533, 0.99903999671936039]


array([0, 0, 0, ..., 9, 9, 9])

In [116]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y, np.argmax(y_pred, axis=1)))
print(confusion_matrix(y, np.argmax(y_pred, axis=1), ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       500
          1       1.00      1.00      1.00       500
          2       1.00      1.00      1.00       500
          3       0.99      0.99      0.99       500
          4       0.99      1.00      1.00       500
          5       0.98      1.00      0.99       500
          6       1.00      1.00      1.00       500
          7       1.00      1.00      1.00       500
          8       1.00      0.99      1.00       500
          9       0.99      0.99      0.99       500

avg / total       1.00      1.00      1.00      5000



/home/disooqi/anaconda2/envs/ztdl/lib/python3.5/site-packages/sklearn/metrics/classification.py:248: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if np.all([l not in y_true for l in labels]):


ValueError: At least one label specified must be in y_true

### Train - Test split

In [36]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold

In [39]:
X_train, X_test, y_train, y_test = train_test_split(features, incidence_y, test_size=0.2)

In [40]:
model.fit(X_train, y_train, epochs=30)   

Epoch 1/30
4000/4000 [==============================] - 0s - loss: 0.0271 - acc: 0.9911     
Epoch 2/30
4000/4000 [==============================] - 0s - loss: 0.0271 - acc: 0.9911     
Epoch 3/30
4000/4000 [==============================] - 0s - loss: 0.0244 - acc: 0.9920     
Epoch 4/30
4000/4000 [==============================] - 0s - loss: 0.0206 - acc: 0.9938     
Epoch 5/30
4000/4000 [==============================] - 0s - loss: 0.0219 - acc: 0.9933     
Epoch 6/30
4000/4000 [==============================] - 0s - loss: 0.0199 - acc: 0.9934     
Epoch 7/30
4000/4000 [==============================] - 0s - loss: 0.0205 - acc: 0.9937     
Epoch 8/30
4000/4000 [==============================] - 0s - loss: 0.0224 - acc: 0.9931     
Epoch 9/30
4000/4000 [==============================] - 0s - loss: 0.0221 - acc: 0.9934     
Epoch 10/30
4000/4000 [==============================] - 0s - loss: 0.0212 - acc: 0.9929     
Epoch 11/30
4000/4000 [==============================] - 0s - loss: 0

In [55]:
model.evaluate(X_test, y_test)
# model.predict(X_test)

  32/1000 [..............................] - ETA: 0s

[0.05270997177809477, 0.98690000724792482]

### NN with Cross-validation 

In [43]:
from keras.wrappers.scikit_learn import KerasClassifier

In [85]:
def build_nn_model():
    model = Sequential()
    model.add(Dense(25, input_shape=(400,), activation='sigmoid')) # sigmoid is better than tanh I don't know why
    # model.add(Dense(2, activation='tanh'))
    model.add(Dense(10, activation='softmax'))  # try sigmoid and softmax
    model.compile(Adam(lr=0.05), 'categorical_crossentropy', metrics=['accuracy'])  # binary_crossentropy
    return model

cv_model = KerasClassifier(build_fn=build_nn_model, epochs=30, verbose=0)

In [89]:
cv = KFold(5,shuffle=True)  # defining a 5-fold cross validation 

scores = cross_val_score(cv_model, features, incidence_y, cv=cv)

print(scores)  # accuracies
print(scores.mean())
print(scores.std())

SyntaxError: invalid syntax (<ipython-input-89-a70d2c35ac11>, line 3)

In [72]:
print(scores)  # accuracies
print(scores.mean())
print(scores.std())

[ 0.97070001  0.96960002  0.97720001  0.97000002  0.97500001]
0.972500012875
0.00304104780247


0.98270001  0.98100002  0.98490001  0.98050002  0.98290001  <br/>
0.982400013161 <br/>
0.00155948217113

[ 0.889  0.91   0.911  0.913  0.911] <br/>
0.9068<br/>
0.00895321171424
